In [2]:
import pandas as pd
df=pd.read_csv(r'D:\DDOS_Detection\ddos_ds\balanced_Ds.csv')

In [8]:
df.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Flow Bytes/s,Flow Packets/s,Packet Length Mean,Packet Length Std,SYN Flag Count,ACK Flag Count,FIN Flag Count,...,Label,Hour,DayOfWeek,Minute,Protocol_6,Protocol_17,Source Port Category_Registered Port,Source Port Category_Well-known Port,Destination Port Category_Registered Port,Destination Port Category_Well-known Port
0,1.666667e-08,0.000010,0.000000,0.000000e+00,2.500000e-01,0.000000,0.000000,0.0,0.0,0.0,...,1,13,5,2,True,False,False,False,False,True
1,4.565250e-04,0.000020,0.001245,9.972647e-05,5.931734e-05,0.285505,0.160342,0.0,0.0,0.0,...,1,13,5,26,True,False,False,False,False,True
2,1.875000e-06,0.000000,0.000249,0.000000e+00,3.333326e-03,0.000000,0.000000,0.0,0.0,0.0,...,1,11,5,8,True,False,False,True,False,False
3,9.599642e-01,0.000300,0.003487,1.528590e-08,1.204136e-07,0.021471,0.059223,0.0,1.0,0.0,...,1,12,5,42,True,False,True,False,False,True
4,2.505827e-01,0.004314,0.219454,9.981927e-05,1.824151e-05,1.000000,1.000000,0.0,1.0,0.0,...,1,9,5,22,True,False,False,False,False,True


In [5]:
df['Label'].value_counts()

Label
1    100000
0    100000
Name: count, dtype: int64

In [6]:
X=df.drop(columns=['Label'])
y=df['Label']
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



In [7]:
X_train.columns

Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Flow Bytes/s', 'Flow Packets/s', 'Packet Length Mean',
       'Packet Length Std', 'SYN Flag Count', 'ACK Flag Count',
       'FIN Flag Count', 'RST Flag Count', 'Flow IAT Mean', 'Flow IAT Std',
       'Hour', 'DayOfWeek', 'Minute', 'Protocol_6', 'Protocol_17',
       'Source Port Category_Registered Port',
       'Source Port Category_Well-known Port',
       'Destination Port Category_Registered Port',
       'Destination Port Category_Well-known Port'],
      dtype='object')

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout


In [16]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32,activation='relu'),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


d:\DDOS_Detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │         1,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,585 (14.00 KB)

 Trainable params: 3,585 (14.00 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
import datetime
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
early_stopping=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_cb=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [21]:
history = model.fit(X_train, y_train, validation_data=[X_val, y_val], epochs=50, callbacks=[early_stopping,tensorflow_cb],batch_size=64)

Epoch 1/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8348 - loss: 0.3728 - val_accuracy: 0.9950 - val_loss: 0.0189
Epoch 2/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9952 - loss: 0.0226 - val_accuracy: 0.9978 - val_loss: 0.0118
Epoch 3/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9973 - loss: 0.0139 - val_accuracy: 0.9983 - val_loss: 0.0104
Epoch 4/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9973 - loss: 0.0142 - val_accuracy: 0.9985 - val_loss: 0.0102
Epoch 5/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9977 - loss: 0.0126 - val_accuracy: 0.9984 - val_loss: 0.0104
Epoch 6/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9977 - loss: 0.0120 - val_accuracy: 0.9985 - val_loss: 0.0101
Epoch 7/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9980 - loss: 0.0104 - val_accuracy: 0.9986 - val_loss: 0.0087
Epoch 8/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9980 - loss: 0.0100 -

In [22]:
loss,accuracy=model.evaluate(X_test,y_test)
print(accuracy)

938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9990 - loss: 0.0049
0.9991666674613953


In [23]:
model.save('D:\DDOS_Detection\models\ddos_detection_model.h5')

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\rohan\AppData\Local\Temp\ipykernel_16376\3791941197.py:1: SyntaxWarning: invalid escape sequence '\D'
  model.save('D:\DDOS_Detection\models\ddos_detection_model.h5')
